This short example uses the ICEYE <a href="https://github.com/iceye-ltd/icecube">ICEcube library</a> and inherents it's dependencies. Install ICEcube using the <a href="https://iceye-ltd.github.io/icecube/installation/">installation instructions</a>.

You can easily download an ICEYE image here: https://www.iceye.com/lp/example-strip-sar-dataset-acre-brazil. This data is a set of time-series data over Acre, Brazil over a forested region of the Amazon.

Learn to uce the ICEcube library to read in the data (at least 1 image), and run a (rather naive and meaingless) kmeans clustering to try to classify like pixels in either the complex or real domain. This is just an exercise to show you how to handle the data arrays and start thinking about techniques you could leverage to assess patterns in the data.

In [ ]:
# Get an ICEYE image or 2 from here: https://www.iceye.com/lp/example-strip-sar-dataset-acre-brazil
# They can be large, so it may take a while 

# Hierarchical Data Format (HDF) of ICEYE SLC imagery
import icecube
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from icecube.bin.sar_cube.slc_datacube import SLCDatacube
from icecube.bin.sar_cube.grd_datacube import GRDDatacube
from icecube.bin.config import CubeConfig 


Put the SLC images in 1 folder and the GRDs in another folder. It can help to get things organized. I only have 1 image in each folder.

```
├── grds
│   └── ICEYE_GRD_SM_71820_20210722T055012.tif
└── slcs
    ├── ICEYE_SLC_SM_71820_20210722T055012.h5
    ├── ICEYE_SLC_SM_71820_20210722T055012.xml
```

In [ ]:
cube_config = CubeConfig()
slc_datacube = SLCDatacube.build(cube_config, 'slcs')
grd_datacube = GRDDatacube.build(cube_config, 'grds')


Each image pixel is represented by a complex (I and Q) magnitude value and therefore contains both amplitude and phase information. The processing for all SLC products results in a single look in each dimension using the full available signal bandwidth. The imagery is geo-referenced using orbit and attitude data from the satellite. SLC images are produced in a zero Doppler geometry. This convention is common with the standard slant range products available from other SAR sensors.



In [ ]:
# What does the data look like?

slc_datacube.xrdataset.head()


In [ ]:
slc_datacube.xrdataset['Real']

In [ ]:
slc_datacube.xrdataset['Complex']

In [ ]:
slc_datacube.xrdataset['Complex'][0]


In [ ]:
grd_datacube.xrdataset.head()


In [ ]:
intensity=grd_datacube.xrdataset['Intensity'][0].values
intensity

These are really big files (GB each!) But we use the memory optimized xarray format to handle this. Visualing can take a while here though, so let's just focus on a small subset. I am going to create a 2k x 2k chip(s).



In [ ]:
# Lets look at an amplitude chunk!
fig, ax = plt.subplots(1,2)
ax[0].imshow(intensity[1000:3000, 1000:3000])
ax[1].imshow(intensity[3001:5001, 3001:5001])

In [ ]:
rl=slc_datacube.xrdataset['Real'][0].values
cmplx=slc_datacube.xrdataset['Complex'][0].values

In [ ]:
# And we can crop the real and complex imagery, similarly to the GRD above.
crop_rl_tr = rl[1000:3000, 1000:3000]
crop_rl_tst = rl[3001:5001, 3001:5001]
crop_cl_tr = cmplx[1000:3000, 1000:3000]
crop_cl_tst = cmplx[3001:5001, 3001:5001]

In [ ]:
#Plot the complex/real image chips

fig, ax = plt.subplots(1,2)
ax[0].imshow(abs(crop_rl_tst))
ax[1].imshow(abs(crop_cl_tst))

Now, let's make a kmeans classifier that attempts to cluster the pixels by type to create a 'poor-mans' semantic segmentor. To use kmeans, we have to typically flatten the 2D array to 1D.

In [ ]:
rl_f_tr=crop_rl_tr.flatten().reshape(-1, 1)     #for k-means must be 1D
rl_f_tst=crop_rl_tst.flatten().reshape(-1, 1)     #for k-means must be 1D

cl_f_tr=crop_cl_tr.flatten().reshape(-1, 1)     #for k-means must be 1D
cl_f_tst=crop_cl_tst.flatten().reshape(-1, 1)     #for k-means must be 1D

in_f_tst=intensity[3001:5001, 3001:5001].flatten().reshape(-1, 1)
#cmplx_f=cmplx.flatten().reshape(-1, 1)

In [ ]:
clt = KMeans(n_clusters = 10,random_state=0)   #no clue how many clusters! I am just lazy and don't want to label the data. 
k=clt.fit(in_f_tst)    #you can change to any of the other data types like the complex data here.


In [ ]:
out=k.fit_predict(in_f_tst)

In [ ]:
from skimage import color
from skimage import io
io.imshow(color.label2rgb(out.reshape(2000,2000)))


We may not have learned anything scientific from the exercise, but now you can perhaps do actual work with these arrays. 

hyvää työtä